In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("../data/clean_sales_data.csv")
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QUARTER,MONTH,YEAR,...,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE,REVENUE
0,10107,30,95.70,2,2871.00,2003-02-24,Shipped,1,2,2003,...,,NYC,NY,10022,USA,not assigned,Yu,Kwai,Small,2871.00
1,10121,34,81.35,5,2765.90,2003-05-07,Shipped,2,5,2003,...,,Reims,unknown,51100,France,EMEA,Henriot,Paul,Small,2765.90
2,10134,41,94.74,2,3884.34,2003-07-01,Shipped,3,7,2003,...,,Paris,unknown,75508,France,EMEA,Da Cunha,Daniel,Medium,3884.34
3,10145,45,83.26,6,3746.70,2003-08-25,Shipped,3,8,2003,...,,Pasadena,CA,90003,USA,not assigned,Young,Julie,Medium,3746.70
4,10159,49,100.00,14,5205.27,2003-10-10,Shipped,4,10,2003,...,,San Francisco,CA,unknown,USA,not assigned,Brown,Julie,Medium,4900.00


In [ ]:
# WE ANALYZED ARE THE SALES IMPROVING , SEASONAL OR ARE DROPPING

monthly_revenue = df.groupby(['YEAR','MONTH'])['REVENUE'].sum().reset_index()

monthly_revenue['YEAR_MONTH'] = monthly_revenue['YEAR'].astype(str) + '-' + monthly_revenue['MONTH'].astype(str)

plt.figure(figsize=(16,9))
plt.plot(monthly_revenue['YEAR_MONTH'], monthly_revenue['REVENUE'])
plt.xticks(rotation=45)
plt.xlabel("MONTH")
plt.ylabel("REVENUE")
plt.title("MONTHLY REVENUE TREND")
plt.tight_layout()
plt.show()


Sales show a consistent seasonal spike between the months of SEPTEMBER and DECEMBER , peaking in OCTOBER and early NOVEMBER . This coincides with major festive periods, where customer spending increases significantly . Post-festival demand decreases mid-November onward,indicating the need for inventory scale up before OCTOBER and controlled stock replenishment afterward .

In [ ]:
# WE WILL SEE WHICH ARE TOP 10 PRODUCTS BY REVENUE

top_products = df.groupby('PRODUCTLINE')['REVENUE'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(14,8))
top_products.plot(kind="bar")
plt.xlabel("PRODUCT CATEGORY")
plt.ylabel("REVENUE")
plt.title("THE TOP 10 PRODUCTS BY REVENUE")
plt.xticks(rotation=40)
plt.tight_layout()
plt.show()

CLASSIC CARS - generated the highest revenue among all the product categories . This suggests strong collector demand and brand value . Expanding CLASSIC CAR variants and offering limited eeditions could increase revenue further.

In [ ]:
# WE WILL SEE THE TOP 10 CUSTOMERS WHO SPENDS THE MOST 

top_customer = df.groupby('CUSTOMERNAME')['REVENUE'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(14,8))
top_customer.plot(kind="bar")
plt.xlabel("CUSTOMER'S NAME")
plt.ylabel("TOTAL SPENT")
plt.xticks(rotation=40)
plt.title("TOP 10 CUSTOMERS BY SPEND")
plt.tight_layout()
plt.show()



## Top Customer Analysis

*Euro Shopping Channel* is the highest revenue-generating customer. This customer consistently places high-value orders, contributing a significant share to overall sales. Their buying volume indicates a strong and stable business relationship.

### Business Insight

Maintaining and strengthening the relationship with Euro Shopping Channel is strategically valuable. Personalized pricing agreements, priority inventory allocation, and early access to new product lines could further increase their lifetime value and ensure long-term retention.

In [ ]:
# WE WILL SEE THE COUNTRY OR REGION THAT HAS THE MOST SALES OR REVENUE

top_countries = df.groupby('COUNTRY')['REVENUE'].sum().sort_values(ascending=False)

plt.figure(figsize=(14,8))
top_countries.head(10).plot(kind="bar")
plt.xlabel("COUNTRIES")
plt.ylabel("REVENUE")
plt.title("THE TOP 10 COUNTRIES WITH THE MOST SALES AND REVENUE")
plt.xticks(rotation=40)
plt.tight_layout()
plt.show()

## Top Country by Revenue

The *United States* is the highest revenue-generating market. Sales volume and customer demand from the USA are significantly higher compared to other regions, indicating strong market presence and brand acceptance.

### Business Insight

Since the USA drives the largest share of total revenue, it should remain a priority market for sales and marketing initiatives. Increasing targeted promotions, optimizing supply chain efficiency, and expanding product availability in this region could further boost revenue growth and strengthen market dominance.

In [ ]:
peak_months = df[df['MONTH'].isin ([9 , 10 , 11 , 12])]

monthly_sales = peak_months.groupby('MONTH')['SALES'].sum().sort_index()

plt.figure(figsize=(10,6))
monthly_sales.plot(kind='bar', color='skyblue')

plt.title("Sales During Peak Months (Sep–Dec)")
plt.xlabel("Month")
plt.ylabel("Total Sales")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


### 🕓 Sales Trends in Peak Months (September–December)

The following visualization focuses on the company's performance during the final four months of the year — **September through December**, which are typically the busiest months for sales.  
We aggregate total sales by month and visualize the results as a bar chart to easily identify which month contributes the most revenue.


In [ ]:
# ANALYSING WHICH PRODUCTS HAD THE HIGHEST DEMAND DURING THE PEAK MONTHS

peak_months = df[df['MONTH'].isin ([9 , 10 , 11 , 12])]

top_prod_during_peak = peak_months.groupby('PRODUCTLINE')['SALES'].sum().sort_values(ascending=False)

plt.figure(figsize=(12,6))
top_products.plot(kind='bar', color='lightcoral')

plt.title("Top 10 Product Lines (Peak Months)")
plt.xlabel("Product Line")
plt.ylabel("Total Sales")
plt.xticks(rotation=40)
plt.tight_layout()
plt.show()

### 💼 Top Performing Product Lines in Peak Months

This chart highlights the **top 10 product lines** that generated the most revenue during the peak months (September–December).  
By aggregating total sales per product line, we can quickly identify which categories drive the highest seasonal performance.


In [10]:
# AVERAGE ORDER VALUE 

revenue_per_order = df.groupby("ORDERNUMBER")['REVENUE'].sum().reset_index()

aov = revenue_per_order['REVENUE'].mean()
print("AVERAGE ORDER VALUE (AOV) : " , round(aov,2))

AVERAGE ORDER VALUE (AOV) :  27231.93


In [15]:
# RFM CALCULATION 

df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'])

# calculating the current date of purchase
latest_date = df['ORDERDATE'].max()

rfm = df.groupby('CUSTOMERNAME').agg({
    "ORDERDATE" : lambda x : (latest_date - x.max()), # recency
    "ORDERNUMBER" : "nunique",  # frequency
    "SALES" : "sum"   # monetary
}).reset_index()

rfm.columns = ['CUSTOMERNAME','Recency','Frequency','Monetary']

rfm.head()

,CUSTOMERNAME,Recency,Frequency,Monetary
0,"AV Stores, Co.",181 days,3,157807.81
1,Alpha Cognac,50 days,3,70488.44
2,Amica Models & Co.,250 days,2,94117.26
3,"Anna's Decorations, Ltd",69 days,4,153996.13
4,Atelier graphique,173 days,3,24179.96


In [18]:
# Giving RFM scores to the customers and segmentation

rfm['R_SCORE'] = pd.qcut(rfm['Recency'] , 5 , labels=[1,2,3,4,5])
rfm['F_SCORE'] = pd.qcut(rfm['Frequency'].rank(method="first") , 5 , labels=[1,2,3,4,5])
rfm['M_SCORE'] = pd.qcut(rfm['Monetary'] ,5 , labels=[1,2,3,4,5])

rfm['RFM_segment'] = rfm['R_SCORE'].astype(str) + rfm['F_SCORE'].astype(str) + rfm['M_SCORE'].astype(str)
rfm['RFM_Score'] = rfm[['R_SCORE', 'F_SCORE', 'M_SCORE']].astype(int).sum(axis=1)


# Segment customers by overall score

rfm['Customer_Type'] = pd.cut(
    rfm['RFM_Score'],
    bins=[0,6,10,15],
    labels=['Low Value','Medium Value','High Value']
)

# Check distribution
rfm['Customer_Type'].value_counts()



Customer_Type
Medium Value    56
High Value      23
Low Value       13
Name: count, dtype: int64

## Customer Segmentation – Insight

The RFM analysis shows clear customer clusters such as Champions, Loyal Customers, Potential Loyalists, and At-Risk customers. 

- *Champions* are the most valuable and engaged customers. They should be prioritized with exclusive offers and early product access.
- *Loyal Customers* consistently buy and respond well to cross-selling.
- *Potential Loyalists* can be nurtured with personalized discounts and engagement campaigns.
- *At-Risk / Dormant* customers need reactivation strategies such as seasonal promotions and reminder outreach.

This segmentation helps in designing targeted marketing and retention strategies rather than treating all customers equally.

In [ ]:
def rfm_segment(row):
    r = int(row['R_SCORE'])
    f = int(row['F_SCORE'])
    m = int(row['M_SCORE'])
    total = int(row['RFM_Score'])
    
    # High engagement and spending
    if total >= 13:
        return 'Champions'
    elif r >= 4 and f >= 3:
        return 'Loyal Customers'
    elif r >= 4 and f <= 2:
        return 'Recent Customers'
    elif r <= 2 and f >= 4:
        return 'At Risk'
    elif total <= 6:
        return 'Lost Customers'
    elif f >= 3 and m >= 3:
        return 'Potential Loyalist'
    else:
        return 'Others'

rfm['Segment'] = rfm.apply(rfm_segment, axis=1)

rfm['Segment'].value_counts()

plt.figure(figsize=(10,6))
rfm['Segment'].value_counts().plot(kind='bar', color='skyblue')

plt.title('Customer Segmentation Based on RFM Scores', fontsize=14, fontweight='bold')
plt.xlabel('Customer Segment')
plt.ylabel('Number of Customers')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


Segment
Recent Customers      30
At Risk               22
Potential Loyalist    17
Lost Customers         9
Others                 6
Champions              5
Loyal Customers        3
Name: count, dtype: int64

### Champions Segment Insight

There are *5 Champion customers* who purchase frequently, spend the most, and have bought recently. This segment is small but extremely valuable. 

These customers should receive:
- Early access to new product launches
- Personalized offers or loyalty benefits
- Priority customer service

Retaining these 5 customers has a disproportionately positive impact on revenue due to their high spending behavior.